# Topic Modeling with Python

Using LDA algorithm in Gensim library to perform topic modeling.

In [1]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [2]:
# nltk stopwords
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words.extend(['use','re','from','subject','edu'])

In [3]:
# import data
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique)
df.head()

<bound method Series.unique of 0                       rec.autos
1           comp.sys.mac.hardware
2           comp.sys.mac.hardware
3                   comp.graphics
4                       sci.space
                   ...           
11309                     sci.med
11310       comp.sys.mac.hardware
11311    comp.sys.ibm.pc.hardware
11312               comp.graphics
11313             rec.motorcycles
Name: target_names, Length: 11314, dtype: object>


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


In [4]:
# remove emails and newline characters

# convert to list
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


In [5]:
# tokenize and clean up text
def sent_to_word(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True)) # deacc=True, remove punctuations

data_words = list(sent_to_word(data))

print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


In [6]:
# create bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


In [7]:
# remove stop-words, create bigrams and lemmatize

# function to remove stop-words
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['where', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'could', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [9]:
# create dictionary and corpus needed for topic modeling

id2word = corpora.Dictionary(data_lemmatized)

# create corpus
texts = data_lemmatized

# term document frequency
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 1), (7, 2), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]]


In [13]:
# build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                             id2word=id2word,
                                             num_topics=20,
                                             random_state=42,
                                             update_every=1,
                                             chunksize=100,
                                             passes=10,
                                             alpha='auto',
                                             per_word_topics=True)

In [14]:
pprint(lda_model.print_topics())

[(0,
  '0.073*"young" + 0.072*"bike" + 0.054*"ride" + 0.038*"weight" + 0.036*"foot" '
  '+ 0.035*"air" + 0.028*"sc" + 0.028*"sport" + 0.027*"rider" + '
  '0.027*"saturn"'),
 (1,
  '0.149*"window" + 0.080*"image" + 0.061*"screen" + 0.057*"display" + '
  '0.055*"graphic" + 0.049*"video" + 0.046*"format" + 0.034*"mouse" + '
  '0.026*"cop" + 0.021*"resolution"'),
 (2,
  '0.136*"space" + 0.037*"orbit" + 0.036*"launch" + 0.034*"mission" + '
  '0.032*"earth" + 0.031*"external" + 0.027*"satellite" + 0.027*"plane" + '
  '0.024*"moon" + 0.023*"hell"'),
 (3,
  '0.047*"would" + 0.033*"know" + 0.030*"be" + 0.027*"think" + 0.022*"make" + '
  '0.021*"good" + 0.018*"get" + 0.018*"want" + 0.017*"could" + 0.017*"well"'),
 (4,
  '0.023*"believe" + 0.023*"people" + 0.021*"say" + 0.021*"evidence" + '
  '0.020*"reason" + 0.014*"claim" + 0.012*"sense" + 0.012*"may" + '
  '0.011*"exist" + 0.011*"fact"'),
 (5,
  '0.163*"book" + 0.066*"page" + 0.065*"draw" + 0.054*"author" + '
  '0.045*"review" + 0.045*"paper" 

In [15]:
doc_lda = lda_model[corpus]

In [16]:
# compute model perplexity and coherence score

print('\nPerplexity: ', lda_model.log_perplexity(corpus))

# coherence score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ',coherence_lda)


Perplexity:  -13.781615726087555

Coherence Score:  0.4539162215350685


In [17]:
# visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.330784 -0.056416       1        1  22.058239
11     0.289731  0.147309       2        1  20.170631
8      0.311703 -0.171574       3        1  17.040733
10     0.231792  0.127584       4        1  12.689601
4      0.309758 -0.124202       5        1  11.329379
15     0.031586  0.340853       6        1   2.774196
16    -0.042993  0.079193       7        1   1.954533
17    -0.024959 -0.044081       8        1   1.495500
9     -0.046241 -0.117493       9        1   1.312350
19    -0.099668 -0.001304      10        1   1.166638
1     -0.112165  0.003927      11        1   1.096195
14    -0.093825  0.028321      12        1   1.084309
7     -0.095648 -0.077196      13        1   0.984778
18    -0.111355 -0.001907      14        1   0.980442
2     -0.118812 -0.016353      15        1   0.785930
5     -0.142364 -0.019877      16        1   0.731203
0     -0.140966 -0.022685      17        1   0.698337
13    -0.151588 -0.018098      18        1   0.651299
12    -0.156601 -0.025362      19        1   0.603610
6     -0.168168 -0.030642      20        1   0.392094, topic_info=             Term          Freq         Total Category  logprob  loglift
18           line  16426.000000  16426.000000  Default  30.0000  30.0000
197         write  13691.000000  13691.000000  Default  29.0000  29.0000
177       article   7991.000000   7991.000000  Default  28.0000  28.0000
325         would  13510.000000  13510.000000  Default  27.0000  27.0000
39           year   4829.000000   4829.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
4083        brain      0.046694      1.270990  Topic20 -11.4938   2.2375
3967         item      0.046694      1.271018  Topic20 -11.4938   2.2375
34017         eeg      0.046693      1.270956  Topic20 -11.4938   2.2375
4274   connection      0.046694      1.270999  Topic20 -11.4938   2.2375
3400          pin      0.046694      1.271043  Topic20 -11.4938   2.2374

[868 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
2885     16  0.977634      abstract
40       18  0.972625  acceleration
1227      5  0.998269        accept
1307     13  0.994987       acquire
1390      5  0.954647           act
...     ...       ...           ...
39        3  0.812130          year
39        4  0.010975          year
39        8  0.158823          year
1027     19  0.976323        yellow
570      17  0.998205         young

[1138 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 12, 9, 11, 5, 16, 17, 18, 10, 20, 2, 15, 8, 19, 3, 6, 1, 14, 13, 7])